# <a name="how"></a> Как сделать пулл реквест?

0. Выбираем, где хотим провести изменения, в форке репозитория (более предпочтительно, но не принципиально) или в самом репозитории (в этом случае нужно запросить у меня доступ).
1. Нужно произвести все желаемые изменения в семинарском ноутбуке. И убедиться, что эти изменения сохранены (юпитер у меня иногда тупит, поэтому жму трижды `ctrl-s` с интервалом около секунды).
  <br> Постарайтесь ограничиться минимальными изменениями. Так же убедитесь, что у вас актуальная версия репозитория. (Я сам в недавние ноутбуки могу теоретически каждый день коммитить, а мои изменения затирать не надо :) ).
2. Далее в этом ноутбуке (он умеет правильно генерировать `.md` файлы):
  <br>A.  <a href="#what" style="color:#856024"> Здесь </a> выбираем семинар(ы), к которому сделали правку. `../tools` выбирать не надо.
  <br>B.  <a href="#github" style="color:#856024"> Здесь </a> можно написать свой commit message, если есть желание. Можно оставить как есть. В этом репозитории нет культуры хороших сообщений к коммитам :)
  <br>C.  Запускаем этот ноутбук, он сгенерит `.md`-шки и закоммитит изменения на гитхаб.
3. Если изменение было в форке, то делаем пулл реквест.

### <a name="what"></a> Выбираем что коммитить

In [102]:
import glob
import os
import subprocess

highlevel_dirs = sum([
    #["../tools"], 
    sorted(glob.glob("../sem28*")),
    #sorted(glob.glob("../sem28*")),
], [])

print("Highlevel dirs:", highlevel_dirs)

Highlevel dirs: ['../sem28-unix-time']


In [103]:
tmp_dir = "./tmp_dir"
get_ipython().system('rm -r {tmp_dir} ; mkdir {tmp_dir} 2>&1 | grep -v "File exists"'.format(tmp_dir=tmp_dir))

### Генерируем все .md-шки стандартными средствами
\+ Делаем .md-шки очищенные для вывода. По этим .md-шкам можно будет смотреть реальную историю изменений. И дифф при пулреквестах.

In [104]:
from multiprocessing import Pool

tasks = []

def convert_tasks(n, d):
    no_output_file = d + "_no_output"
    src_copy = str(abs(hash(n))) + '_' + os.path.basename(n)
    path = os.path.dirname(n)
    return [
        "jupyter nbconvert {} --to markdown --output {}".format(n, d),
        " && ".join([
            "cp {src} {tmp_dir}/{src_copy}",
            "jupyter nbconvert {tmp_dir}/{src_copy} --ClearOutputPreprocessor.enabled=True --inplace",
            "jupyter nbconvert {tmp_dir}/{src_copy} --to markdown --output {src_copy}",
            "cp {tmp_dir}/{src_copy}.md {path}/{no_output_file}.md",
        ]).format(src=n, no_output_file=no_output_file, dst=d, tmp_dir=tmp_dir, src_copy=src_copy, path=path),
    ]
    
for subdir in highlevel_dirs:
    notebooks = glob.glob(subdir + "/*.ipynb")
    print(subdir, notebooks)
    for m in glob.glob(subdir + "/*.md"):
        os.remove(m)
    if len(notebooks) == 1:
        tasks.extend(convert_tasks(notebooks[0], "README"))
    else:
        for n in notebooks:
            tasks.extend(convert_tasks(n, os.path.basename(n.replace(".ipynb", ""))))
        nmds = [os.path.basename(n).replace(".ipynb", ".md") for n in notebooks]
        with open(os.path.join(subdir, "README.md"), "w") as f:
            f.write('\n'.join(
                ['# Ноутбуки семинара'] + 
                ['* [{nmd}]({nmd})'.format(nmd=nmd) for nmd in nmds] + 
                ['']
            ))

print("\n".join(tasks))

def execute_all_in_parallel(tasks):
    ps = []
    for t in tasks:
        ps.append(subprocess.Popen(["bash", "-c", t], stdout=subprocess.PIPE, stderr=subprocess.PIPE))
    for p in ps:
        out, err = p.communicate()
        print(out.decode(), err.decode())

execute_all_in_parallel(tasks)

../sem28-unix-time ['../sem28-unix-time/time.ipynb']
jupyter nbconvert ../sem28-unix-time/time.ipynb --to markdown --output README
cp ../sem28-unix-time/time.ipynb ./tmp_dir/3597715960520351318_time.ipynb && jupyter nbconvert ./tmp_dir/3597715960520351318_time.ipynb --ClearOutputPreprocessor.enabled=True --inplace && jupyter nbconvert ./tmp_dir/3597715960520351318_time.ipynb --to markdown --output README_no_output && cp ./tmp_dir/README_no_output.md ../sem28-unix-time
 [NbConvertApp] Converting notebook ../sem28-unix-time/time.ipynb to markdown
[NbConvertApp] Writing 32749 bytes to ../sem28-unix-time/README.md

 [NbConvertApp] Converting notebook ./tmp_dir/3597715960520351318_time.ipynb to notebook
[NbConvertApp] Writing 35263 bytes to ./tmp_dir/3597715960520351318_time.ipynb
[NbConvertApp] Converting notebook ./tmp_dir/3597715960520351318_time.ipynb to markdown
[NbConvertApp] Writing 28740 bytes to ./tmp_dir/README_no_output.md



### Магические исправления

Стандартная конвертилка не учитывает некоторых особенностей маркдауна на гитхабе и некоторых особенностей моих ноутбуков. Поэтому результат надо допилить напильником.

In [105]:
import re


def basic_improve(fname):
    with open(fname, "r") as f:
        r = f.read()
    for b in ["\x00", "\x1B", "\x08"]:
        r = r.replace(b, "")
    with open(fname, "w") as f:
        f.write(r)
    return "dos2unix {}".format(fname)

def improve_md(fname):
    with open(fname, "r") as f:
        r = f.read()
    r = r.replace("```python\n%%cpp", "```cpp\n%%cpp")
    r = r.replace("```python\n%%cmake", "```cmake\n%%cmake")
    r = r.replace('\n', "SUPER_SLASH" + "_N_REPLACER")
    
    r = re.sub(r'\<\!--MD_BEGIN_FILTER--\>.*?\<\!--MD_END_FILTER--\>', "", r)
    #r = re.sub(r'(\#SET_UP_MAGIC_BEGIN.*?\#SET_UP_MAGIC_END)', "<too much code>", r)
    r = re.sub(r'\<\!\-\-\ YANDEX_METRICA_BEGIN\ \-\-\>.*\<\!\-\-\ YANDEX_METRICA_END\ \-\-\>', '', r)
    
    r = r.replace("<IPython.core.display.Javascript object>", '')
    r = r.replace("SUPER_SLASH" + "_N_REPLACER", '\n')
    
    template = "#""MAGICS_SETUP_END"
    bpos = r.rfind(template)
    if bpos != -1:
        r = r[bpos + len(template):]
        template = "```"
        bpos = r.find(template)
        assert bpos >= 0
        r = r[bpos + len(template):]
    
    
    template = "<""!-- MAGICS_SETUP_PRINTING_END -->"
    bpos = r.rfind(template)
    if bpos != -1:
        r = r[bpos + len(template):]
    
    def file_repl(matchobj, path=os.path.dirname(fname)):
        fname = os.path.join(path, matchobj.group(1))
        if fname.find("__FILE__") == -1:
            with open(fname, "r") as f:
                return "\n```\n" + f.read() + "\n```\n"
    
    r = r.replace("</td>", "")
    r = r.replace("</tr>", "")
    
    r = re.sub(r'\<\!--MD_FROM_FILE (.*?) --\>', file_repl, r)
    with open(fname, "w") as f:
        f.write(r)
        
def improve_file(fname):
    if fname.endswith(".md"):
        improve_md(fname)


In [106]:
tasks = []
shell_tasks = []


for sfx in [".ipynb", ".md"]:
    for hdir in highlevel_dirs:
        for fname in glob.glob("./{}/*".format(hdir) + sfx):
            shell_tasks.append(basic_improve(fname))
            tasks.append(lambda fname=fname: improve_file(fname))
            
execute_all_in_parallel(shell_tasks)
for t in tasks:
    t()

 dos2unix: converting file ./../sem28-unix-time/time.ipynb to Unix format...

 dos2unix: converting file ./../sem28-unix-time/README.md to Unix format...

 dos2unix: converting file ./../sem28-unix-time/README_no_output.md to Unix format...



### Смотрим изменения

In [107]:
for subdir in highlevel_dirs:
    get_ipython().system("git diff {}/*_no_output*".format(subdir))

diff --git a/sem28-unix-time/README_no_output.md b/sem28-unix-time/README_no_output.md
index f804aef..634cb76 100644
--- a/sem28-unix-time/README_no_output.md
+++ b/sem28-unix-time/README_no_output.md
@@ -1,488 +1,370 @@
 
 
-# Динамические библиотеки
+# Опрос для всех, кто зашел на эту страницу
 
+Он не страшный, там всего два обязательных вопроса на выбор одного варианта из трёх. Извиняюсь за размер, но к сожалению студенты склонны игнорировать опросы :| 
+
+Пытаюсь компенсировать :)
+
+<a href="https://docs.google.com/forms/d/e/1FAIpQLSdUnBAae8nwdSduZieZv7uatWPOMv9jujCM4meBZcHlTikeXg/viewform?usp=sf_link"><img src="poll.png" width="100%"  align="left" alt="Опрос"></a>
+
+
+
+# Работа со временем в С/С++
+
+Поговорим о типах времени в C/C++ и функциях для получения текущего времени, парсинга из строк, сериализации в строки.
+
+Меня всегда дико напрягало отсутствие одного хорошего типа времени, наличие времени в разных часовых поясах и куча разных типов сериализации. Постараюсь собрат

### <a name="github"></a> Коммитим на github

In [92]:
cmds = []
add_cmd = "git add --ignore-errors "
add_cmd_f = "git add --ignore-errors -f "
for subdir in highlevel_dirs:
    for sfx in [".ipynb", ".md", ".c", ".cpp"]:
        cmds.append(add_cmd + " {}/*{}".format(subdir, sfx))
    cmds.append(add_cmd_f + " -f {}/bash_popen_tmp/*.html".format(subdir))
    cmds.append(add_cmd_f + " -f {}/interactive_launcher_tmp/*.log".format(subdir))
    cmds.append("git add -u {}".format(subdir))
    
def execute_cmd(cmd):
    print(">", cmd)
    get_ipython().system(cmd)
    
for cmd in cmds:
    execute_cmd(cmd)
# execute_cmd("git add -u")
execute_cmd("git commit -m 'yet another update'")
execute_cmd("git push origin master")

> git add --ignore-errors  ../sem01/*.ipynb
> git add --ignore-errors  ../sem01/*.md
> git add --ignore-errors  ../sem01/*.c
> git add --ignore-errors  ../sem01/*.cpp
fatal: pathspec '../sem01/*.cpp' did not match any files
> git add --ignore-errors -f  -f ../sem01/bash_popen_tmp/*.html
fatal: pathspec '../sem01/bash_popen_tmp/*.html' did not match any files
> git add --ignore-errors -f  -f ../sem01/interactive_launcher_tmp/*.log
fatal: pathspec '../sem01/interactive_launcher_tmp/*.log' did not match any files
> git add -u ../sem01
> git add --ignore-errors  ../sem03-ints-floats/*.ipynb
> git add --ignore-errors  ../sem03-ints-floats/*.md
> git add --ignore-errors  ../sem03-ints-floats/*.c
> git add --ignore-errors  ../sem03-ints-floats/*.cpp
fatal: pathspec '../sem03-ints-floats/*.cpp' did not match any files
> git add --ignore-errors -f  -f ../sem03-ints-floats/bash_popen_tmp/*.html
fatal: pathspec '../sem03-ints-floats/bash_popen_tmp/*.html' did not match any files
> git add --ignor

> git add --ignore-errors -f  -f ../sem11-mmap-instrumentation/interactive_launcher_tmp/*.log
fatal: pathspec '../sem11-mmap-instrumentation/interactive_launcher_tmp/*.log' did not match any files
> git add -u ../sem11-mmap-instrumentation
> git add --ignore-errors  ../sem12-fork-exec-pipe/*.ipynb
> git add --ignore-errors  ../sem12-fork-exec-pipe/*.md
> git add --ignore-errors  ../sem12-fork-exec-pipe/*.c
> git add --ignore-errors  ../sem12-fork-exec-pipe/*.cpp
> git add --ignore-errors -f  -f ../sem12-fork-exec-pipe/bash_popen_tmp/*.html
fatal: pathspec '../sem12-fork-exec-pipe/bash_popen_tmp/*.html' did not match any files
> git add --ignore-errors -f  -f ../sem12-fork-exec-pipe/interactive_launcher_tmp/*.log
fatal: pathspec '../sem12-fork-exec-pipe/interactive_launcher_tmp/*.log' did not match any files
> git add -u ../sem12-fork-exec-pipe
> git add --ignore-errors  ../sem13-signal/*.ipynb
> git add --ignore-errors  ../sem13-signal/*.md
> git add --ignore-errors  ../sem13-signal/*.

> git add -u ../sem21-ipc-synchronizing
> git add --ignore-errors  ../sem22-dynamic-lib/*.ipynb
> git add --ignore-errors  ../sem22-dynamic-lib/*.md
> git add --ignore-errors  ../sem22-dynamic-lib/*.c
> git add --ignore-errors  ../sem22-dynamic-lib/*.cpp
> git add --ignore-errors -f  -f ../sem22-dynamic-lib/bash_popen_tmp/*.html
fatal: pathspec '../sem22-dynamic-lib/bash_popen_tmp/*.html' did not match any files
> git add --ignore-errors -f  -f ../sem22-dynamic-lib/interactive_launcher_tmp/*.log
fatal: pathspec '../sem22-dynamic-lib/interactive_launcher_tmp/*.log' did not match any files
> git add -u ../sem22-dynamic-lib
> git add --ignore-errors  ../sem23-extra-net-protocols/*.ipynb
> git add --ignore-errors  ../sem23-extra-net-protocols/*.md
> git add --ignore-errors  ../sem23-extra-net-protocols/*.c
> git add --ignore-errors  ../sem23-extra-net-protocols/*.cpp
fatal: pathspec '../sem23-extra-net-protocols/*.cpp' did not match any files
> git add --ignore-errors -f  -f ../sem23-extra

> git push origin master
Enumerating objects: 125, done.
Counting objects: 100% (125/125), done.
Delta compression using up to 4 threads
Compressing objects: 100% (76/76), done.
Writing objects: 100% (78/78), 72.20 KiB | 1.76 MiB/s, done.
Total 78 (delta 51), reused 0 (delta 0)
remote: Resolving deltas: 100% (51/51), completed with 35 local objects.
To github.com:yuri-pechatnov/caos_2019-2020.git
   0c5902e..40c557d  master -> master
